### **Bài thực hành 3: Bắt đầu với Hugging Face**
**Tên sinh viên:** [Tên của bạn]
**Mã số sinh viên:** [MSSV của bạn]
**Lớp:** [Lớp của bạn]

---
### **Exercise 2: Tinh chỉnh (Finetuning) một mô hình đã được huấn luyện trước cho bài toán phân loại văn bản nhị phân**

Trong bài tập này, chúng ta sẽ thực hiện một quy trình hoàn chỉnh để tinh chỉnh một mô hình ngôn ngữ đã được huấn luyện trước (pre-trained model) cho một tác vụ cụ thể: phân loại cảm xúc trên bộ dữ liệu đánh giá phim IMDB. Đây là một ví dụ kinh điển của bài toán phân loại văn bản nhị phân (tích cực/tiêu cực).

#### **Bước 1 & 2: Tải thư viện và bộ dữ liệu**

Đầu tiên, chúng ta cần các thư viện `datasets` để tải dữ liệu và `evaluate` để tính toán các độ đo. Bộ dữ liệu "imdb" là một bộ dữ liệu tiêu chuẩn chứa các bài đánh giá phim và nhãn tương ứng (0 cho tiêu cực, 1 cho tích cực).

Để quá trình huấn luyện diễn ra nhanh chóng, chúng ta sẽ chỉ lấy một tập con nhỏ của dữ liệu để làm ví dụ.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Tải bộ dữ liệu IMDB
print("Đang tải bộ dữ liệu IMDB...")
dataset = load_dataset("imdb")
print("Tải dữ liệu thành công!")

# Để tiết kiệm thời gian, chúng ta sẽ tạo một tập dữ liệu con nhỏ hơn
# Lấy 1000 mẫu cho tập huấn luyện và 500 mẫu cho tập kiểm tra
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(500))

print("\nKích thước tập huấn luyện con:", len(small_train_dataset))
print("Kích thước tập kiểm tra con:", len(small_test_dataset))
print("\nVí dụ một mẫu dữ liệu:")
print(small_train_dataset[0])

#### **Bước 3: Tải mô hình Pre-trained và Tokenizer**

Chúng ta sẽ sử dụng `distilbert-base-uncased`, một phiên bản nhỏ hơn và nhanh hơn của BERT, rất phù hợp cho việc tinh chỉnh. `AutoTokenizer` sẽ tự động tải tokenizer tương ứng với mô hình, và `AutoModelForSequenceClassification` sẽ tải kiến trúc mô hình với một lớp phân loại ở trên cùng.

In [ ]:
# Tên của mô hình pre-trained
model_name = "distilbert-base-uncased"

# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tải mô hình với 2 nhãn (tiêu cực, tích cực)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

print("Đã tải xong tokenizer và mô hình.")

#### **Bước 4: Tiền xử lý dữ liệu**

Mô hình không thể hiểu văn bản thô. Chúng ta cần chuyển đổi văn bản thành các con số mà mô hình có thể xử lý. Quá trình này được gọi là **tokenization**. Chúng ta sẽ tạo một hàm để tokenize văn bản và áp dụng nó cho toàn bộ bộ dữ liệu.

In [ ]:
# Tạo hàm tiền xử lý
def preprocess_function(examples):
    # Tokenize văn bản, cắt bớt nếu dài hơn 512 token
    return tokenizer(examples["text"], truncation=True)

# Áp dụng hàm tiền xử lý cho tập train và test
tokenized_train_dataset = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = small_test_dataset.map(preprocess_function, batched=True)

print("\nMột mẫu dữ liệu sau khi tokenize:")
print(tokenized_train_dataset[0])

#### **Bước 5 & 6: Định nghĩa tham số huấn luyện và tạo đối tượng Trainer**

`TrainingArguments` là một lớp chứa tất cả các siêu tham số cho quá trình huấn luyện, chẳng hạn như learning rate, số epoch, kích thước batch,...

`Trainer` là một lớp API cấp cao của Hugging Face giúp đơn giản hóa vòng lặp huấn luyện và đánh giá.

In [ ]:
# Định nghĩa các tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",              # Thư mục lưu kết quả
    evaluation_strategy="epoch",         # Đánh giá sau mỗi epoch
    learning_rate=2e-5,                  # Tốc độ học
    per_device_train_batch_size=16,      # Kích thước batch cho tập train
    per_device_eval_batch_size=16,       # Kích thước batch cho tập test
    num_train_epochs=3,                  # Số epoch huấn luyện
    weight_decay=0.01,                   # Giảm trọng số để tránh overfitting
)

# Tải độ đo accuracy
metric = evaluate.load("accuracy")

# Định nghĩa hàm tính toán độ đo
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Tạo đối tượng Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
)

# Bắt đầu quá trình fine-tuning
print("\nBắt đầu quá trình tinh chỉnh mô hình...")
trainer.train()
print("Hoàn tất tinh chỉnh!")

#### **Bước 7: Đánh giá mô hình đã được tinh chỉnh**

Sau khi huấn luyện, chúng ta cần đánh giá hiệu suất của mô hình trên tập dữ liệu kiểm tra (test set) để xem nó hoạt động tốt đến mức nào trên dữ liệu chưa từng thấy.

In [ ]:
# Thực hiện đánh giá trên tập test
print("\nĐang đánh giá mô hình trên tập kiểm tra...")
eval_results = trainer.evaluate()

print("\n--- Kết quả đánh giá ---")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")
print("------------------------")

# Lấy dự đoán trên tập test để vẽ ma trận nhầm lẫn
predictions = trainer.predict(tokenized_test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = tokenized_test_dataset["label"]

# Tính toán ma trận nhầm lẫn
cm = confusion_matrix(true_labels, predicted_labels)

# Vẽ ma trận nhầm lẫn
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Tiêu cực', 'Tích cực'], yticklabels=['Tiêu cực', 'Tích cực'])
plt.xlabel('Nhãn dự đoán')
plt.ylabel('Nhãn thực tế')
plt.title('Ma trận nhầm lẫn (Confusion Matrix)')
plt.show()

**Báo cáo và phân tích kết quả:**

1.  **Hiệu suất mô hình:** Dựa vào kết quả từ hàm `evaluate()`, chúng ta có thể thấy độ chính xác (accuracy) của mô hình trên tập dữ liệu kiểm tra. Một mô hình được tinh chỉnh tốt thường sẽ đạt độ chính xác cao (ví dụ: trên 85-90% cho bài toán này, ngay cả với một tập dữ liệu con). Điều này cho thấy mô hình đã học được cách phân biệt giữa các bài đánh giá phim tích cực và tiêu cực.

2.  **Phân tích Ma trận nhầm lẫn (Confusion Matrix):**
    *   **True Positives (TP)** và **True Negatives (TN)** (các giá trị trên đường chéo chính) cho biết số lượng mẫu được mô hình phân loại đúng. Giá trị càng cao càng tốt.
    *   **False Positives (FP)** và **False Negatives (FN)** (các giá trị không nằm trên đường chéo chính) biểu thị số lượng mẫu bị phân loại sai. Giá trị càng thấp càng tốt.
    *   Nhìn vào ma trận, chúng ta có thể đánh giá xem mô hình có xu hướng nhầm lẫn loại cảm xúc nào hơn. Ví dụ, nếu số FN cao, nghĩa là mô hình thường dự đoán sai các mẫu tích cực thành tiêu cực.

**Kết luận:**
Qua bài tập này, chúng ta đã thực hiện thành công việc tinh chỉnh một mô hình ngôn ngữ lớn cho tác vụ phân loại văn bản nhị phân. Bắt đầu từ một mô hình `distilbert-base-uncased` có kiến thức tổng quát, chúng ta đã "chuyên môn hóa" nó để hiểu sắc thái trong các bài đánh giá phim và đạt được hiệu suất phân loại tốt trên dữ liệu thực tế. Quy trình này cho thấy sức mạnh của học chuyển giao (transfer learning) trong lĩnh vực xử lý ngôn ngữ tự nhiên.
